# Scenariusze testowe dla porównania wydajności baz danych

### 1. Operacja CREATE

- Dodanie nowego nauczyciela
- Utworzenie nowej klasy
- Dodanie nowego przedmiotu
- Zarejestrowanie nowego ucznia
- Przypisanie ucznia do klasy (**Dodano: Zapisanie ucznia do klasy (enrolment)**)
- Utworzenie harmonogramu zajęć
- Wystawienie oceny

### 2. Operacja READ

Pobranie kompleksowego raportu zawierającego:
- Dane osobowe ucznia
- Informacje o klasie (**Dodano: Informacje o zapisach do klas**)
- Dane nauczyciela prowadzącego
- Listę ocen z opisami przedmiotów
- Szczegółowy harmonogram zajęć

### 3. Operacja UPDATE

- Aktualizacja danych ucznia
- Zmiana przypisania do klasy (**Dodano: Aktualizacja zapisu do klasy**)
- Modyfikacja nazwy klasy
- Aktualizacja danych nauczyciela
- Zmiana oceny
- Aktualizacja opisu przedmiotu
- Modyfikacja harmonogramu zajęć

### 4. Operacja DELETE

- Usunięcie ocen ucznia
- Wypisanie ucznia z klasy (**Dodano: Usunięcie zapisu do klasy**)
- Usunięcie harmonogramu zajęć
- Usunięcie klasy
- Opcjonalne usunięcie przedmiotów
- Opcjonalne usunięcie nauczyciela
- Usunięcie rekordu ucznia

## Ilość rekordów do testów

Testy będą przeprowadzane dla następujących ilości rekordów:

1. 10,000 rekordów
2. 100,000 rekordów
3. 1,000,000 rekordów
4. 10,000,000 rekordów

## Metryki wydajnościowe

Dla każdego scenariusza i ilości rekordów będziemy mierzyć:

1. Czas wykonania całego scenariusza
2. Średni czas pojedynczych operacji
3. Liczbę operacji na sekundę (throughput)
4. Zużycie zasobów systemowych (CPU, RAM, I/O dysku)

# Narzędzia i technologie testowe

### Wbudowane instrumenty bazodanowe

Każdy system oferuje specjalizowane narzędzia diagnostyczne:

| System | Narzędzie | Funkcjonalności |
| :-- | :-- | :-- |
| PostgreSQL | pgBench | Testy TPC-B, własne skrypty SQL |
| MariaDB | sysbench | Testy OLTP, skalowanie pionowe |
| MongoDB | mongoperf | Operacje na dokumentach JSON |
| Cassandra | cassandra-stress | Testy dystrybucji danych |
| Redis | redis-benchmark | Pomiar opóźnień operacji klucz-wartość |

Wykorzystanie natywnych narzędzi pozwala na precyzyjne badanie specyficznych mechanizmów storage engine.

### Automatyzacja w Pythonie

Kluczowe biblioteki wspierające testy:

- **SQLAlchemy** dla baz relacyjnych
- **PyMongo** dla MongoDB
- **Cassandra-driver** dla Cassandra
- **redis-py** dla Redis

In [3]:
# Import required libraries
import psycopg2
import psycopg2.errors
from pymongo import MongoClient
from cassandra.cluster import Cluster
import redis
import mysql.connector
import yaml
import pandas as pd
import os
import time
import sys
from pathlib import Path

# Load database configuration
print("Setting up database connections...")
with open('docker-compose.yml', 'r') as file:
    docker_config = yaml.safe_load(file)

# PostgreSQL connection
postgres_config = docker_config['services']['postgresql']
postgres_client = psycopg2.connect(
    host='localhost',
    database=postgres_config['environment']['POSTGRES_DB'],
    user=postgres_config['environment']['POSTGRES_USER'],
    password=postgres_config['environment']['POSTGRES_PASSWORD'],
    port=postgres_config['ports'][0].split(':')[0]
)

# MariaDB connection
mariadb_config = docker_config['services']['mariadb']
mariadb_client = mysql.connector.connect(
    host='localhost',
    database=mariadb_config['environment']['MYSQL_DATABASE'],
    user=mariadb_config['environment']['MYSQL_USER'],
    password=mariadb_config['environment']['MYSQL_PASSWORD'],
    port=mariadb_config['ports'][0].split(':')[0],
    allow_local_infile=True
)

# MongoDB connection
mongo_config = docker_config['services']['mongodb']
mongo_client = MongoClient(
    host='localhost',
    port=int(mongo_config['ports'][0].split(':')[0])
)

# Cassandra connection
cassandra_config = docker_config['services']['cassandra']
cassandra_client = Cluster(['localhost'], port=cassandra_config['ports'][0].split(':')[0])
cassandra_session = cassandra_client.connect()

# Redis connection
redis_config = docker_config['services']['redis']
redis_client = redis.Redis(
    host='localhost',
    port=int(redis_config['ports'][0].split(':')[0])
)

# Test connections
try:
    postgres_client.cursor().execute("SELECT 1")
    print("INFO: PostgreSQL connection successful")
    
    mariadb_client.cursor(buffered=True).execute("SELECT 1")
    print("INFO: MariaDB connection successful")
    
    cassandra_session.execute("SELECT release_version FROM system.local")
    print("INFO: Cassandra connection successful")
    
    mongo_client.admin.command('ping')
    print("INFO: MongoDB connection successful")
    
    redis_client.ping()
    print("INFO: Redis connection successful")
except Exception as e:
    print(f"ERROR: Connection test failed: {e}")

Setting up database connections...
INFO: PostgreSQL connection successful
INFO: MariaDB connection successful
INFO: Cassandra connection successful
INFO: MongoDB connection successful
INFO: Redis connection successful
INFO: PostgreSQL connection successful
INFO: MariaDB connection successful
INFO: Cassandra connection successful
INFO: MongoDB connection successful
INFO: Redis connection successful


In [4]:
CELL_END = ''

In [5]:
# Data generation functions
sys.path.append(str(Path.cwd()))
from generator import generate_school_data

def generate_files(output_dir='./data', scale=1000, batch_size=10000, **kwargs):
    """
    Generate synthetic school data files for benchmarking.
    """
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    print(f"INFO: Generating data with scale {scale} and batch size {batch_size}...")

    result = generate_school_data(
        output_dir=output_dir,
        scale=scale,
        batch_size=batch_size,
        **kwargs
    )

    print(f"INFO: Generated {len(result['students'])} students, {len(result['teachers'])} teachers, " + 
          f"{len(result['classes'])} classes, {len(result['subjects'])} subjects")
    print("="*50)
    return result

# Generate test data sets
scale_100_dir = './data/scale_100'
scale_1000_dir = './data/scale_1000'

generate_files(output_dir=scale_100_dir, scale=100, batch_size=5000)
generate_files(output_dir=scale_1000_dir, scale=1000, batch_size=5000)
CELL_END


INFO: Generating data with scale 100 and batch size 5000...
Processing students.csv...
  Batch 1/1 processed for students.csv
Processing teachers.csv...
  Batch 1/1 processed for teachers.csv
Processing classes.csv...
  Batch 1/1 processed for classes.csv
Processing subjects.csv...
  Batch 1/1 processed for subjects.csv
Processing grades.csv...
  Batch 1/1 processed for grades.csv
Processing schedules.csv...
  Batch 1/1 processed for schedules.csv
Processing enrollments.csv...
  Batch 1/1 processed for enrollments.csv
INFO: Generated 1000 students, 100 teachers, 200 classes, 100 subjects
INFO: Generating data with scale 1000 and batch size 5000...
Processing students.csv...
  Batch 1/2 processed for students.csv
  Batch 2/2 processed for students.csv
Processing teachers.csv...
  Batch 1/1 processed for teachers.csv
Processing classes.csv...
  Batch 1/1 processed for classes.csv
Processing subjects.csv...
  Batch 1/1 processed for subjects.csv
Processing grades.csv...
  Batch 1/6 proces

''

# PostgreSQL Operations

In [6]:
# PostgreSQL Methods

def initialize_postgres_schema(conn, schema_sql):
    """
    Initializes the PostgreSQL database schema using the provided SQL script.
    """
    if not schema_sql:
        print("ERROR: Schema SQL content is empty.")
        return

    try:
        with conn.cursor() as cur:
            cur.execute(schema_sql)
        conn.commit()
        print("INFO: PostgreSQL schema initialized.")
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error initializing PostgreSQL schema: {e}")

def verify_postgres_tables(conn, expected_tables):
    """
    Verifies if the expected tables exist in PostgreSQL.
    """
    try:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public' AND table_name = ANY(%s);
            """, (expected_tables,))
            existing_tables = {row[0] for row in cur.fetchall()}

        missing_tables = set(expected_tables) - existing_tables
        if not missing_tables:
            print(f"INFO: All PostgreSQL tables exist: {', '.join(expected_tables)}")
            return True
        else:
            print(f"WARNING: Missing PostgreSQL tables: {', '.join(missing_tables)}")
            return False
    except Exception as e:
        print(f"ERROR: Error verifying PostgreSQL tables: {e}")
        return False

def insert_postgres_table_from_csv(conn, table_name, csv_file) -> tuple[float, float, float]:
    # Inserts data from a CSV file into a PostgreSQL table using INSERT.
    # Does not fail on duplicate key errors.
    # Assumes the table already exists and has the same structure as the CSV file.
    
    operation_start_time = time.time() # Initialize start_time
    file_opened_start_time = 0 # Initialize file_opened_start_time
    try:
        with conn.cursor() as cur:
            with open(csv_file, 'r') as f:
                next(f)  # Skip header
                file_opened_start_time = time.time() # Initialize start_time when file is opened
                for line in f:
                    values = line.strip().split(',')
                    insert_sql = f"INSERT INTO {table_name} VALUES ({', '.join(['%s'] * len(values))})"
                    try:
                        cur.execute(insert_sql, values)
                    except psycopg2.errors.UniqueViolation:
                        # Ignore duplicate key errors
                        conn.rollback()
                        continue
                    except Exception as e:
                        print(f"ERROR: Error inserting into {table_name}: {e}")
                        conn.rollback()
                        break
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error inserting data from {csv_file} into {table_name}: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)

def copy_postgres_table_from_csv(conn, table_name, csv_file) -> tuple[float, float, float]:
    # Inserts data from a CSV file into a PostgreSQL table using COPY.
    # Does not fail on duplicate key errors.
    # Assumes the table already exists and has the same structure as the CSV file.
    operation_start_time = time.time() # Initialize start_time
    file_opened_start_time = 0 # Initialize file_opened_start_time
    try:
        with conn.cursor() as cur:
            copy_sql = f"COPY {table_name} FROM STDIN WITH (FORMAT CSV, HEADER)"
            with open(csv_file, 'r') as f:
                file_opened_start_time = time.time() # Initialize start_time when file is opened
                cur.copy_expert(sql=copy_sql, file=f)
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error inserting data from {csv_file} into {table_name}: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)

def copy_postgres_enrollments_from_csv(conn, csv_file):
    # This function is a specialized version for the enrollments table, because it has a composite primary key.
    # Inserts data from a CSV file into the enrollments table using COPY.
    # Does not fail on duplicate key errors.
    # Uses a temporary table to handle duplicates.

    operation_start_time = time.time() # Initialize start_time
    file_opened_start_time = 0 # Initialize file_opened_start_time

    try:
        with conn.cursor() as cur:
            # Create a temporary table for the COPY operation
            temp_table_name = "temp_enrollments"
            cur.execute(f"""
                CREATE TEMP TABLE {temp_table_name} (
                    student_id INT,
                    class_id INT,
                    enrolled_at TIMESTAMP
                ) ON COMMIT DROP;
            """)
            copy_sql = f"COPY {temp_table_name} FROM STDIN WITH (FORMAT CSV, HEADER)"
            with open(csv_file, 'r') as f:
                file_opened_start_time = time.time()
                cur.copy_expert(sql=copy_sql, file=f)

            # Insert into the main table, ignoring duplicates
            insert_sql = f"""
                INSERT INTO enrollments (student_id, class_id, enrolled_at)
                SELECT student_id, class_id, enrolled_at FROM {temp_table_name}
                ON CONFLICT (student_id, class_id) DO NOTHING;
            """
            cur.execute(insert_sql)
            conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error inserting data from {csv_file} into enrollments: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)

def load_postgres_data(conn, data_dir):
    """
    Loads data from CSV files into PostgreSQL tables.
    """
    data_path = Path(data_dir)
    table_csv_map = {
        'teachers': 'teachers.csv',
        'subjects': 'subjects.csv',
        'classes': 'classes.csv',
        'students': 'students.csv',
        'grades': 'grades.csv',
        'schedules': 'schedules.csv',
        # 'enrollments': 'enrollments.csv' # Handled separately
    }
    for table_name, csv_file in table_csv_map.items():
        op_time, f_op_time, end_time = copy_postgres_table_from_csv(conn, table_name, data_path / csv_file)
        print(f"INFO: Inserted {table_name} in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")

    # Handle enrollments separately due to composite primary key
    op_time, f_op_time, end_time = copy_postgres_enrollments_from_csv(conn, data_path / 'enrollments.csv')
    print(f"INFO: Inserted enrollments in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")

def verify_postgres_counts(conn, tables):
    """
    Counts rows in PostgreSQL tables.
    """
    counts = {}
    max_len = max(len(t) for t in tables) if tables else 0
    print(f"INFO: Counting rows in PostgreSQL tables")
    try:
        with conn.cursor() as cur:
            for table_name in tables:
                try:
                    cur.execute(f"SELECT COUNT(*) FROM {table_name};")
                    count = cur.fetchone()[0]
                    counts[table_name] = count
                except Exception as count_error:
                    print(f"ERROR: {count_error}")
                    counts[table_name] = 'Error'

        print("--- PostgreSQL Table Row Counts ---")
        for table, count in counts.items():
            print(f"{table:<{max_len}} : {count}")
        print("-----------------------------------")
        return counts

    except Exception as e:
        print(f"ERROR: {e}")
        return None

In [7]:
# PostgreSQL Operations Execution

# Schema initialization
with open('schemas/postgres_schema.sql', 'r') as f:
    sql_schema = f.read()

initialize_postgres_schema(postgres_client, sql_schema)

# Table verification 
required_tables = ['teachers', 'subjects', 'classes', 'students', 'enrollments', 'grades', 'schedules']
verify_postgres_tables(postgres_client, required_tables)

# Data loading
load_postgres_data(postgres_client, scale_100_dir)

# Count verification
verify_postgres_counts(postgres_client, required_tables)
CELL_END

INFO: PostgreSQL schema initialized.
INFO: All PostgreSQL tables exist: teachers, subjects, classes, students, enrollments, grades, schedules
INFO: Inserted teachers in 0.00 seconds (file opened in 0.00 seconds)
INFO: Inserted subjects in 0.00 seconds (file opened in 0.00 seconds)
INFO: Inserted classes in 0.00 seconds (file opened in 0.00 seconds)
INFO: Inserted students in 0.00 seconds (file opened in 0.00 seconds)
INFO: Inserted grades in 0.02 seconds (file opened in 0.02 seconds)
INFO: Inserted schedules in 0.00 seconds (file opened in 0.00 seconds)
INFO: Inserted enrollments in 0.02 seconds (file opened in 0.02 seconds)
INFO: Counting rows in PostgreSQL tables
--- PostgreSQL Table Row Counts ---
teachers    : 100
subjects    : 100
classes     : 200
students    : 1000
enrollments : 1994
grades      : 3000
schedules   : 500
-----------------------------------


''

# MariaDB Operations

In [8]:
# MariaDB Methods

def initialize_mariadb_schema(conn, schema_sql):
    """
    Initializes the MariaDB database schema using the provided SQL script.
    """
    if not schema_sql:
        print("ERROR: Schema SQL content is empty.")
        return
    try:
        with conn.cursor() as cur:
            for statement in schema_sql.split(';'):
                stmt = statement.strip()
                if stmt:
                    cur.execute(stmt)
        conn.commit()
        print("INFO: MariaDB schema initialized.")
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error initializing MariaDB schema: {e}")

def verify_mariadb_tables(conn, expected_tables):
    """
    Verifies if the expected tables exist in MariaDB.
    """
    try:
        with conn.cursor() as cur:
            format_strings = ','.join(['%s'] * len(expected_tables))
            cur.execute(f"""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = DATABASE() AND table_name IN ({format_strings});
            """, tuple(expected_tables))
            existing_tables = {row[0] for row in cur.fetchall()}

        missing_tables = set(expected_tables) - existing_tables
        if not missing_tables:
            print(f"INFO: All MariaDB tables exist: {', '.join(expected_tables)}")
            return True
        else:
            print(f"WARNING: Missing MariaDB tables: {', '.join(missing_tables)}")
            return False
    except Exception as e:
        print(f"ERROR: Error verifying MariaDB tables: {e}")
        return False

def insert_mariadb_table_from_csv(conn, table_name, csv_file) -> tuple[float, float, float]:
    """Inserts data from a CSV file into a MariaDB table by reading the header for columns."""
    operation_start_time = time.time()
    file_opened_start_time = 0
    try:
        with conn.cursor() as cur:
            with open(csv_file, 'r') as f:
                # read header for column names
                header = next(f).strip().split(',')
                cols = header
                placeholders = ','.join(['%s'] * len(cols))
                insert_sql = f"INSERT INTO {table_name} ({','.join(cols)}) VALUES ({placeholders})"
                file_opened_start_time = time.time()
                for line in f:
                    values = line.strip().split(',')
                    # ensure values length matches columns
                    if len(values) != len(cols):
                        if len(values) > len(cols):
                            values = values[:len(cols)]
                        else:
                            print(f"WARNING: Skipping {table_name} row with {len(values)} values (expected {len(cols)}, values: {values})")
                            continue
                    try:
                        cur.execute(insert_sql, values)
                    except mysql.connector.errors.IntegrityError:
                        conn.rollback()
                        continue
                    except Exception as e:
                        print(f"ERROR: Error inserting into {table_name}: {e}")
                        conn.rollback()
                        break
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error inserting data from {csv_file} into {table_name}: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)


def copy_mariadb_table_from_csv(conn, table_name, csv_file) -> tuple[float, float, float]:
    # Inserts data from a CSV file into a MariaDB table using COPY.
    # Does not fail on duplicate key errors.
    # Assumes the table already exists and has the same structure as the CSV file.
    
    operation_start_time = time.time() # Initialize start_time
    file_opened_start_time = 0 # Initialize file_opened_start_time
    try:
        with conn.cursor() as cur:
            copy_sql = f"""
            LOAD DATA LOCAL INFILE '{csv_file}'
            INTO TABLE {table_name}
            FIELDS TERMINATED BY ','
            OPTIONALLY ENCLOSED BY '"'
            LINES TERMINATED BY '\n'
            IGNORE 1 LINES;
            """
            with open(csv_file, 'r') as f:
                file_opened_start_time = time.time() # Initialize start_time when file is opened
                cur.execute(copy_sql)
        conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error inserting data from {csv_file} into {table_name}: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)

def copy_mariadb_enrollments_from_csv(conn, csv_file):
    # This function is a specialized version for the enrollments table, because it has a composite primary key.
    # Inserts data from a CSV file into the enrollments table using COPY.
    # Does not fail on duplicate key errors.
    # Uses a temporary table to handle duplicates.

    operation_start_time = time.time() # Initialize start_time
    file_opened_start_time = 0 # Initialize file_opened_start_time

    try:
        with conn.cursor() as cur:
            # Handle enrollments with INSERT IGNORE to skip duplicates
            print(f"INFO: Loading enrollments with duplicate handling...")
            with open(csv_file, 'r') as f:
                next(f)  # skip header
                for line in f:
                    student_id, class_id, enrolled_at = line.strip().split(',')
                    cur.execute(
                        """
                        INSERT IGNORE INTO enrollments (student_id, class_id, enrolled_at)
                        VALUES (%s, %s, %s)
                        """,
                        (student_id, class_id, enrolled_at)
                    )
            conn.commit()
    except Exception as e:
        conn.rollback()
        print(f"ERROR: Error inserting data from {csv_file} into enrollments: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)

def load_mariadb_data(conn, data_dir):
    table_csv_map = {
    'teachers': 'teachers.csv',
    'subjects': 'subjects.csv',
    'classes': 'classes.csv',
    'students': 'students.csv',
    'grades': 'grades.csv',
    'schedules': 'schedules.csv',
    # 'enrollments': 'enrollments.csv' Handled separately
    }
    data_path = Path(data_dir)
    for table_name, csv_file in table_csv_map.items():
        op_time, f_op_time, end_time = insert_mariadb_table_from_csv(conn, table_name, data_path / csv_file)
        print(f"INFO: Inserted {table_name} in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")

    # Handle enrollments separately due to composite primary key
    op_time, f_op_time, end_time = copy_mariadb_enrollments_from_csv(conn, data_path / 'enrollments.csv')
    print(f"INFO: Inserted enrollments in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")

def verify_mariadb_counts(conn, tables):
    """
    Counts rows in MariaDB tables.
    """
    counts = {}
    max_len = max(len(t) for t in tables) if tables else 0
    print(f"INFO: Counting rows in MariaDB tables")
    
    try:
        with conn.cursor() as cur:
            for table_name in tables:
                try:
                    cur.execute(f"SELECT COUNT(*) FROM {table_name};")
                    count = cur.fetchone()[0]
                    counts[table_name] = count
                except Exception as count_error:
                    print(f"ERROR: {count_error}")
                    counts[table_name] = 'Error'

        print("--- MariaDB Table Row Counts ---")
        for table, count in counts.items():
            print(f"{table:<{max_len}} : {count}")
        print("---------------------------------")
        return counts

    except Exception as e:
        print(f"ERROR: {e}")
        return None

In [9]:
# MariaDB Operations Execution

# Schema initialization
with open('schemas/mariadb_schema.sql', 'r') as f:
    mariadb_schema = f.read()

initialize_mariadb_schema(mariadb_client, mariadb_schema)

# Table verification
required_tables = ['teachers', 'subjects', 'classes', 'students', 'enrollments', 'grades', 'schedules']
verify_mariadb_tables(mariadb_client, required_tables)

# Data loading
load_mariadb_data(mariadb_client, scale_100_dir)

# Count verification
verify_mariadb_counts(mariadb_client, required_tables)
CELL_END

INFO: MariaDB schema initialized.
INFO: All MariaDB tables exist: teachers, subjects, classes, students, enrollments, grades, schedules
INFO: Inserted teachers in 0.01 seconds (file opened in 0.01 seconds)
INFO: Inserted subjects in 0.01 seconds (file opened in 0.01 seconds)
INFO: Inserted classes in 0.03 seconds (file opened in 0.03 seconds)
INFO: Inserted students in 0.13 seconds (file opened in 0.13 seconds)
INFO: Inserted grades in 0.38 seconds (file opened in 0.38 seconds)
INFO: Inserted schedules in 0.06 seconds (file opened in 0.06 seconds)
INFO: Loading enrollments with duplicate handling...
INFO: Inserted enrollments in 0.27 seconds (file opened in 1746445937.27 seconds)
INFO: Counting rows in MariaDB tables
--- MariaDB Table Row Counts ---
teachers    : 100
subjects    : 100
classes     : 200
students    : 1000
enrollments : 1994
grades      : 3000
schedules   : 500
---------------------------------


''

# MongoDB Operations

In [10]:
# MongoDB Methods
def initialize_mongo_schema(client, db_name='benchmark'):
    """
    Initializes the MongoDB schema by creating necessary collections.
    """
    try:
        db = client[db_name]
        
        # List of collections to create based on no_sql_design.txt
        collections = ['students', 'teachers', 'classes', 'subjects']
        
        # Drop existing collections if they exist
        for collection in collections:
            if collection in db.list_collection_names():
                db[collection].drop()
                print(f"INFO: Dropped MongoDB collection: {collection}")
        
        # Create collections with indexes
        for collection in collections:
            db.create_collection(collection)
            print(f"INFO: Created MongoDB collection: {collection}")
            
            # Create indexes for performance
            if collection == 'students':
                db[collection].create_index([("last_name", 1), ("first_name", 1)])
            elif collection == 'classes':
                db[collection].create_index([("name", 1)])
                
        print("INFO: MongoDB schema initialized.")
    except Exception as e:
        print(f"ERROR: {e}")

def verify_mongo_collections(client, db_name='benchmark', expected_collections=None):
    """
    Verifies if the expected collections exist in MongoDB.
    """
    if expected_collections is None:
        expected_collections = ['students', 'teachers', 'classes', 'subjects']
    
    try:
        db = client[db_name]
        existing_collections = db.list_collection_names()
        
        missing_collections = set(expected_collections) - set(existing_collections)
        if not missing_collections:
            print(f"INFO: All MongoDB collections exist: {', '.join(expected_collections)}")
            return True
        else:
            print(f"WARNING: Missing MongoDB collections: {', '.join(missing_collections)}")
            return False
    except Exception as e:
        print(f"ERROR: {e}")
        return False

def insert_mongo_data_from_csv(client, collection_name, csv_file) -> tuple[float, float, float]:
    operation_start_time = time.time() # Initialize start_time
    file_opened_start_time = 0 # Initialize file_opened_start_time
    try:
        db = client['benchmark']
        collection = db[collection_name]
        
        with open(csv_file, 'r') as f:
            reader = pd.read_csv(f)
            # rename id to _id for MongoDB
            if 'id' in reader.columns:
                reader.rename(columns={'id': '_id'}, inplace=True)

            file_opened_start_time = time.time() # Initialize start_time just before starting to insert
            for _, row in reader.iterrows():
                doc = row.to_dict()
                collection.insert_one(doc)
    except Exception as e:
        print(f"ERROR: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, file_opened_start_time, end_time)

def insert_mongo_students_from_csv(client, data_path) -> tuple[float, float]:
    # load all grades and enrollments into students from csv files
    # create a student object with embedded enrollments and grades
    students_file = data_path / 'students.csv'
    enrollments_file = data_path / 'enrollments.csv'
    grades_file = data_path / 'grades.csv'
    operation_start_time = time.time() # Initialize start_time

    try:
        db = client['benchmark']
        collection = db['students']
        
        with open(students_file, 'r') as f:
            reader = pd.read_csv(f)
            for _, row in reader.iterrows():
                student_doc = {
                    "_id": row['id'],
                    "first_name": row['first_name'],
                    "last_name": row['last_name'],
                    "birth_date": row['birth_date'],
                    "enrollments": [],
                    "grades": []
                }
                collection.insert_one(student_doc)

        with open(enrollments_file, 'r') as f:
            reader = pd.read_csv(f)
            for _, row in reader.iterrows():
                student_id = row['student_id']
                enrollment_doc = {
                    "class_id": row['class_id'],
                    "enrolled_at": row['enrolled_at']
                }
                collection.update_one(
                    {"_id": student_id},
                    {"$push": {"enrollments": enrollment_doc}}
                )

        with open(grades_file, 'r') as f:
            reader = pd.read_csv(f)
            for _, row in reader.iterrows():
                student_id = row['student_id']
                grade_doc = {
                    "subject_id": row['subject_id'],
                    "grade": row['grade'],
                    "created_at": row['created_at']
                }
                collection.update_one(
                    {"_id": student_id},
                    {"$push": {"grades": grade_doc}}
                )

    except Exception as e:
        print(f"ERROR: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, end_time)


def insert_mongo_classes_from_csv(client, data_path) -> tuple[float, float]:
    # load all teachers and schedules into classes from csv files
    # create a class object with embedded teachers and schedules
    classes_file = data_path / 'classes.csv'
    schedules_file = data_path / 'schedules.csv'
    operation_start_time = time.time() # Initialize start_time

    try:
        db = client['benchmark']
        collection = db['classes']

        with open(classes_file, 'r') as f:
            reader = pd.read_csv(f)
            for _, row in reader.iterrows():
                class_doc = {
                    "_id": row['id'],
                    "name": row['name'],
                    "teacher_id": row['teacher_id'],
                    "schedule": []
                }
                collection.insert_one(class_doc)

        with open(schedules_file, 'r') as f:
            reader = pd.read_csv(f)
            for _, row in reader.iterrows():
                class_id = row['class_id']
                schedule_doc = {
                    "subject_id": row['subject_id'],
                    "day_of_week": row['day_of_week'],
                    "time_start": row['time_start'],
                    "time_end": row['time_end']
                }
                collection.update_one(
                    {"_id": class_id},
                    {"$push": {"schedule": schedule_doc}}
                )

    except Exception as e:
        print(f"ERROR: {e}")
    finally:
        end_time = time.time()
        return (operation_start_time, end_time)

def load_mongo_data(client, data_dir):
    data_path = Path(data_dir)
    insert_mongo_data_from_csv(client, 'teachers', data_path / 'teachers.csv')
    insert_mongo_data_from_csv(client, 'subjects', data_path / 'subjects.csv')
    insert_mongo_students_from_csv(client, data_path)
    insert_mongo_classes_from_csv(client, data_path)

def verify_mongo_counts(client, db_name='benchmark'):
    """
    Counts documents in MongoDB collections.
    """
    collections = ['students', 'teachers', 'classes', 'subjects']
    max_len = max(len(c) for c in collections)
    
    try:
        db = client[db_name]
        counts = {}
        
        for collection in collections:
            try:
                count = db[collection].count_documents({})
                counts[collection] = count
            except Exception as e:
                print(f"ERROR: {e}")
                counts[collection] = 'Error'
                
        print("--- MongoDB Collection Document Counts ---")
        for collection, count in counts.items():
            print(f"{collection:<{max_len}} : {count}")
        print("-----------------------------------------")

        # Additional checks for embedded documents
        try:
            students_with_enrollments = db.students.count_documents({"enrollments": {"$exists": True, "$ne": []}})
            students_with_grades = db.students.count_documents({"grades": {"$exists": True, "$ne": []}})
            classes_with_schedules = db.classes.count_documents({"schedule": {"$exists": True, "$ne": []}})
            
            print("\n--- MongoDB Embedded Document Counts ---")
            print(f"Students with enrollments : {students_with_enrollments}")
            print(f"Students with grades      : {students_with_grades}")
            print(f"Classes with schedules    : {classes_with_schedules}")
            print("-----------------------------------------")
        except Exception as e:
            print(f"ERROR: {e}")
        
        return counts
    except Exception as e:
        print(f"ERROR: {e}")
        return None

In [11]:
# MongoDB Operations Execution

# Schema initialization
initialize_mongo_schema(mongo_client)

# Collection verification
verify_mongo_collections(mongo_client)

# Data loading
load_mongo_data(mongo_client, scale_100_dir)

# Document count verification
verify_mongo_counts(mongo_client)
CELL_END

INFO: Dropped MongoDB collection: students
INFO: Dropped MongoDB collection: teachers
INFO: Dropped MongoDB collection: classes
INFO: Dropped MongoDB collection: subjects
INFO: Created MongoDB collection: students
INFO: Created MongoDB collection: teachers
INFO: Created MongoDB collection: classes
INFO: Created MongoDB collection: subjects
INFO: MongoDB schema initialized.
INFO: All MongoDB collections exist: students, teachers, classes, subjects
--- MongoDB Collection Document Counts ---
students : 1000
teachers : 100
classes  : 200
subjects : 100
-----------------------------------------

--- MongoDB Embedded Document Counts ---
Students with enrollments : 867
Students with grades      : 944
Classes with schedules    : 189
-----------------------------------------


''

In [12]:
# Cassandra data loading functions with minimal memory usage

def initialize_cassandra_schema(session, keyspace='benchmark'):
    """Initializes the Cassandra schema by creating necessary keyspace and tables."""
    try:
        # Create keyspace if not exists
        session.execute(f"""
            CREATE KEYSPACE IF NOT EXISTS {keyspace} 
            WITH REPLICATION = {{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }};
        """)
        
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Drop existing tables if they exist
        tables = ['teachers', 'subjects', 'classes', 'students', 
                    'enrollments', 'grades', 'schedules', 
                    'student_enrollments', 'student_grades']
        
        for table in tables:
            session.execute(f"DROP TABLE IF EXISTS {table};")
            print(f"INFO: Dropped Cassandra table: {table}")
        
        # Create tables with appropriate data types
        session.execute("""
            CREATE TABLE teachers (
                id INT PRIMARY KEY,
                first_name TEXT,
                last_name TEXT
            );
        """)
        
        session.execute("""
            CREATE TABLE subjects (
                id INT PRIMARY KEY,
                name TEXT,
                description TEXT
            );
        """)
        
        session.execute("""
            CREATE TABLE classes (
                id INT PRIMARY KEY,
                name TEXT,
                teacher_id INT
            );
        """)
        
        session.execute("""
            CREATE TABLE students (
                id INT PRIMARY KEY,
                first_name TEXT,
                last_name TEXT,
                birth_date TEXT
            );
        """)
        
        session.execute("""
            CREATE TABLE enrollments (
                student_id INT,
                class_id INT,
                enrolled_at TIMESTAMP,
                PRIMARY KEY (student_id, class_id)
            );
        """)
        
        session.execute("""
            CREATE TABLE grades (
                id INT PRIMARY KEY,
                student_id INT,
                subject_id INT,
                grade FLOAT,
                created_at TIMESTAMP
            );
        """)
        
        session.execute("""
            CREATE TABLE schedules (
                id INT PRIMARY KEY,
                class_id INT,
                subject_id INT,
                day_of_week INT,
                time_start TEXT,
                time_end TEXT
            );
        """)
        
        session.execute("""
            CREATE TABLE student_enrollments (
                student_id INT,
                class_id INT,
                class_name TEXT,
                teacher_id INT,
                enrolled_at TIMESTAMP,
                PRIMARY KEY (student_id, class_id)
            );
        """)
        
        session.execute("""
            CREATE TABLE student_grades (
                student_id INT,
                subject_id INT,
                subject_name TEXT,
                grade FLOAT,
                created_at TIMESTAMP,
                PRIMARY KEY (student_id, subject_id, created_at)
            ) WITH CLUSTERING ORDER BY (subject_id ASC, created_at DESC);
        """)
        
        print("INFO: Cassandra schema initialized.")
    except Exception as e:
        print(f"ERROR: {e}")

def verify_cassandra_tables(session, keyspace='benchmark', expected_tables=None):
    """Verifies if the expected tables exist in Cassandra."""
    if expected_tables is None:
        expected_tables = ['teachers', 'subjects', 'classes', 'students', 
                            'enrollments', 'grades', 'schedules']
    
    try:
        # Get existing tables
        query = f"""
            SELECT table_name FROM system_schema.tables 
            WHERE keyspace_name = '{keyspace}';
        """
        rows = session.execute(query)
        existing_tables = {row.table_name for row in rows}
        
        missing_tables = set(expected_tables) - existing_tables
        if not missing_tables:
            print(f"INFO: All Cassandra tables exist: {', '.join(expected_tables)}")
            return True
        else:
            print(f"WARNING: Missing Cassandra tables: {', '.join(missing_tables)}")
            return False
    except Exception as e:
        print(f"ERROR: {e}")
        return False

def insert_cassandra_teachers(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert teacher data from CSV, line by line."""
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO teachers (id, first_name, last_name) VALUES (?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 3:  # Ensure we have enough columns
                    session.execute(prepared_stmt, [
                        int(values[0]),       # id
                        values[1],            # first_name
                        values[2]             # last_name
                    ])
        
        print("INFO: Inserted teachers successfully")
    except Exception as e:
        print(f"ERROR: Failed to load teachers: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def insert_cassandra_subjects(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert subject data from CSV, line by line."""
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO subjects (id, name, description) VALUES (?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 3:  # Ensure we have enough columns
                    session.execute(prepared_stmt, [
                        int(values[0]),       # id
                        values[1],            # name
                        values[2]             # description
                    ])
        
        print("INFO: Inserted subjects successfully")
    except Exception as e:
        print(f"ERROR: Failed to load subjects: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def insert_cassandra_classes(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert class data from CSV, line by line."""
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO classes (id, name, teacher_id) VALUES (?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 3:  # Ensure we have enough columns
                    session.execute(prepared_stmt, [
                        int(values[0]),       # id
                        values[1],            # name
                        int(values[2])        # teacher_id
                    ])
        
        print("INFO: Inserted classes successfully")
    except Exception as e:
        print(f"ERROR: Failed to load classes: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def insert_cassandra_students(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert student data from CSV, line by line."""
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO students (id, first_name, last_name, birth_date) VALUES (?, ?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 4:  # Ensure we have enough columns
                    session.execute(prepared_stmt, [
                        int(values[0]),       # id
                        values[1],            # first_name
                        values[2],            # last_name
                        values[3]             # birth_date
                    ])
        
        print("INFO: Inserted students successfully")
    except Exception as e:
        print(f"ERROR: Failed to load students: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def insert_cassandra_enrollments(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert enrollment data from CSV, line by line, with timestamp handling."""
    from datetime import datetime
    
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO enrollments (student_id, class_id, enrolled_at) VALUES (?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 3:  # Ensure we have enough columns
                    # Convert timestamp string to datetime object
                    enrolled_at = datetime.fromisoformat(values[2].replace('Z', '+00:00'))
                    
                    session.execute(prepared_stmt, [
                        int(values[0]),       # student_id
                        int(values[1]),       # class_id
                        enrolled_at           # enrolled_at as datetime
                    ])
        
        print("INFO: Inserted enrollments successfully")
    except Exception as e:
        print(f"ERROR: Failed to load enrollments: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def insert_cassandra_grades(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert grades data from CSV, line by line, with UUID and timestamp handling."""
    from datetime import datetime
    from uuid import uuid4
    
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO grades (id, student_id, subject_id, grade, created_at) VALUES (?, ?, ?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 4:  # Ensure we have enough columns
                    # Convert timestamp string to datetime object
                    created_at = datetime.fromisoformat(values[4].replace('Z', '+00:00'))
                    
                    session.execute(prepared_stmt, [
                        int(values[0]),              # id (generated UUID)
                        int(values[1]),       # student_id
                        int(values[2]),       # subject_id
                        float(values[3]),     # grade
                        created_at            # created_at as datetime
                    ])
        
        print("INFO: Inserted grades successfully")
    except Exception as e:
        print(f"ERROR: Failed to load grades: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def insert_cassandra_schedules(session, csv_file, keyspace='benchmark') -> tuple:
    """Insert schedule data from CSV, line by line, with UUID and day mapping."""
    from uuid import uuid4
    
    operation_start_time = time.time()
    file_opened_start_time = 0
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # Day name to integer mapping
        day_map = {
            'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 
            'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7
        }
        
        # Prepare the insert statement
        prepared_stmt = session.prepare(
            "INSERT INTO schedules (id, class_id, subject_id, day_of_week, time_start, time_end) VALUES (?, ?, ?, ?, ?, ?)"
        )
        
        # Process CSV file line by line
        with open(csv_file, 'r') as f:
            # Skip header
            header = next(f)
            file_opened_start_time = time.time()
            
            for line in f:
                values = line.strip().split(',')
                if len(values) >= 6:  # Ensure we have enough columns
                    # Convert day name to integer
                    day_num = day_map.get(values[3], 0)
                    
                    session.execute(prepared_stmt, [
                        int(values[0]),              # id (generated UUID)
                        int(values[1]),       # class_id
                        int(values[2]),       # subject_id
                        day_num,              # day_of_week as int
                        values[4],            # time_start
                        values[5]             # time_end
                    ])
        
        print("INFO: Inserted schedules successfully")
    except Exception as e:
        print(f"ERROR: Failed to load schedules: {e}")
    
    end_time = time.time()
    return (operation_start_time, file_opened_start_time, end_time)

def populate_cassandra_denormalized_tables(session, data_dir, keyspace='benchmark'):
    """
    Populate denormalized tables for efficient queries.
    This requires more memory as we need to join data in Python.
    """
    from datetime import datetime
    import pandas as pd
    
    data_path = Path(data_dir)
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        
        # --- Populate student_enrollments table ---
        # This requires joining enrollments with classes
        enrollments_df = pd.read_csv(data_path / 'enrollments.csv')
        classes_df = pd.read_csv(data_path / 'classes.csv')
        
        # Prepare statement
        stmt = session.prepare(
            "INSERT INTO student_enrollments (student_id, class_id, class_name, teacher_id, enrolled_at) VALUES (?, ?, ?, ?, ?)"
        )
        
        # Join and process
        merged = pd.merge(enrollments_df, classes_df, left_on='class_id', right_on='id')
        for _, row in merged.iterrows():
            enrolled_at = datetime.fromisoformat(row['enrolled_at'].replace('Z', '+00:00'))
            
            session.execute(stmt, [
                int(row['student_id']),
                int(row['class_id']),
                row['name'],
                int(row['teacher_id']),
                enrolled_at
            ])
        
        print("INFO: Populated student_enrollments denormalized table")
        
        # --- Populate student_grades table ---
        # This requires joining grades with subjects
        grades_df = pd.read_csv(data_path / 'grades.csv')
        subjects_df = pd.read_csv(data_path / 'subjects.csv')
        
        # Prepare statement
        stmt = session.prepare(
            "INSERT INTO student_grades (student_id, subject_id, subject_name, grade, created_at) VALUES (?, ?, ?, ?, ?)"
        )
        
        # Join and process
        merged = pd.merge(grades_df, subjects_df, left_on='subject_id', right_on='id')
        for _, row in merged.iterrows():
            created_at = datetime.fromisoformat(row['created_at_x'].replace('Z', '+00:00'))
            
            session.execute(stmt, [
                int(row['student_id']),
                int(row['subject_id']),
                row['name'],
                float(row['grade']),
                created_at
            ])
        
        print("INFO: Populated student_grades denormalized table")
        
    except Exception as e:
        print(f"ERROR: Failed to populate denormalized tables: {e}")

def load_cassandra_data(session, data_dir, keyspace='benchmark'):
    """Load all data into Cassandra tables."""
    data_path = Path(data_dir)
    
    # Use the keyspace
    session.execute(f"USE {keyspace};")
    
    # Insert basic entities
    op_time, f_op_time, end_time = insert_cassandra_teachers(session, data_path / 'teachers.csv')
    print(f"INFO: Inserted teachers in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    op_time, f_op_time, end_time = insert_cassandra_subjects(session, data_path / 'subjects.csv')
    print(f"INFO: Inserted subjects in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    op_time, f_op_time, end_time = insert_cassandra_classes(session, data_path / 'classes.csv')
    print(f"INFO: Inserted classes in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    op_time, f_op_time, end_time = insert_cassandra_students(session, data_path / 'students.csv')
    print(f"INFO: Inserted students in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    # Insert relationships and complex data
    op_time, f_op_time, end_time = insert_cassandra_enrollments(session, data_path / 'enrollments.csv')
    print(f"INFO: Inserted enrollments in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    op_time, f_op_time, end_time = insert_cassandra_grades(session, data_path / 'grades.csv')
    print(f"INFO: Inserted grades in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    op_time, f_op_time, end_time = insert_cassandra_schedules(session, data_path / 'schedules.csv')
    print(f"INFO: Inserted schedules in {end_time - op_time:.2f} seconds (file opened in {end_time - f_op_time:.2f} seconds)")
    
    # Populate denormalized tables
    populate_cassandra_denormalized_tables(session, data_path)

def verify_cassandra_counts(session, keyspace='benchmark'):
    """Count rows in all Cassandra tables."""
    tables = ['teachers', 'subjects', 'classes', 'students', 
                'enrollments', 'grades', 'schedules', 
                'student_enrollments', 'student_grades']
    max_len = max(len(t) for t in tables)
    
    try:
        # Use the keyspace
        session.execute(f"USE {keyspace};")
        counts = {}
        
        for table in tables:
            try:
                rows = session.execute(f"SELECT COUNT(*) FROM {table}")
                count = rows.one()[0]
                counts[table] = count
            except Exception as e:
                print(f"ERROR: {e}")
                counts[table] = 'Error'
        
        print("--- Cassandra Table Row Counts ---")
        for table, count in counts.items():
            print(f"{table:<{max_len}} : {count}")
        print("----------------------------------")
        
        return counts
    except Exception as e:
        print(f"ERROR: {e}")
        return None

In [13]:
# Cassandra Operations Execution

# Schema initialization
initialize_cassandra_schema(cassandra_session)

# Table verification
required_tables = ['teachers', 'subjects', 'classes', 'students', 'enrollments', 'grades', 'schedules']
verify_cassandra_tables(cassandra_session, expected_tables=required_tables)

# Data loading
load_cassandra_data(cassandra_session, scale_100_dir)

# Row count verification
verify_cassandra_counts(cassandra_session)
CELL_END

INFO: Dropped Cassandra table: teachers
INFO: Dropped Cassandra table: subjects
INFO: Dropped Cassandra table: classes
INFO: Dropped Cassandra table: students
INFO: Dropped Cassandra table: enrollments
INFO: Dropped Cassandra table: grades
INFO: Dropped Cassandra table: schedules
INFO: Dropped Cassandra table: student_enrollments
INFO: Dropped Cassandra table: student_grades
INFO: Cassandra schema initialized.
INFO: All Cassandra tables exist: teachers, subjects, classes, students, enrollments, grades, schedules
INFO: Inserted teachers successfully
INFO: Inserted teachers in 0.19 seconds (file opened in 0.18 seconds)
INFO: Inserted subjects successfully
INFO: Inserted subjects in 0.24 seconds (file opened in 0.23 seconds)
INFO: Inserted classes successfully
INFO: Inserted classes in 0.53 seconds (file opened in 0.52 seconds)
INFO: Inserted students successfully
INFO: Inserted students in 2.58 seconds (file opened in 2.57 seconds)
INFO: Inserted enrollments successfully
INFO: Inserted e

''

In [14]:
def postgres_query(conn, query, params=None, fetch_size=None):
    """
    Execute a generic SELECT query on PostgreSQL.
    
    Args:
        conn: PostgreSQL connection object
        query: SQL query string
        params: Optional parameters for the query (list, tuple or dict)
        fetch_size: Number of rows to fetch at once (None for all)
        
    Returns:
        dict: {'data': results, 'time': execution_time, 'row_count': count}
    """
    start_time = time.time()
    results = []
    row_count = 0
    
    try:
        with conn.cursor() as cur:
            cur.execute(query, params or ())
            
            if fetch_size:
                # Fetch in batches
                while True:
                    rows = cur.fetchmany(fetch_size)
                    if not rows:
                        break
                    results.extend([dict(zip([col[0] for col in cur.description], row)) for row in rows])
                    row_count += len(rows)
            else:
                # Fetch all at once
                rows = cur.fetchall()
                results = [dict(zip([col[0] for col in cur.description], row)) for row in rows]
                row_count = len(results)
                
        end_time = time.time()
        return {
            'data': results,
            'time': end_time - start_time,
            'row_count': row_count
        }
    except Exception as e:
        print(f"ERROR: PostgreSQL query failed: {e}")
        return {
            'data': [],
            'time': time.time() - start_time,
            'row_count': 0,
            'error': str(e)
        }

In [15]:
def mariadb_query(conn, query, params=None, fetch_size=None):
    """
    Execute a generic SELECT query on MariaDB.
    
    Args:
        conn: MariaDB connection object
        query: SQL query string
        params: Optional parameters for the query (list, tuple or dict)
        fetch_size: Number of rows to fetch at once (None for all)
        
    Returns:
        dict: {'data': results, 'time': execution_time, 'row_count': count}
    """
    start_time = time.time()
    results = []
    row_count = 0
    
    try:
        with conn.cursor(dictionary=True) as cur:
            cur.execute(query, params or ())
            
            if fetch_size:
                # Fetch in batches
                while True:
                    rows = cur.fetchmany(fetch_size)
                    if not rows:
                        break
                    results.extend(rows)
                    row_count += len(rows)
            else:
                # Fetch all at once
                results = cur.fetchall()
                row_count = len(results)
                
        end_time = time.time()
        return {
            'data': results,
            'time': end_time - start_time,
            'row_count': row_count
        }
    except Exception as e:
        print(f"ERROR: MariaDB query failed: {e}")
        return {
            'data': [],
            'time': time.time() - start_time,
            'row_count': 0,
            'error': str(e)
        }

In [16]:
def mongo_query(client, collection_name, query=None, projection=None, sort=None, limit=None, db_name='benchmark'):
    """
    Execute a query (equivalent to SELECT) on MongoDB.
    
    Args:
        client: MongoDB client object
        collection_name: Name of the collection to query
        query: MongoDB query dictionary (equivalent to WHERE)
        projection: Fields to include/exclude (equivalent to SELECT columns)
        sort: Sorting specification (equivalent to ORDER BY)
        limit: Maximum number of documents to return (equivalent to LIMIT)
        db_name: Database name
        
    Returns:
        dict: {'data': results, 'time': execution_time, 'row_count': count}
    """
    start_time = time.time()
    results = []
    
    try:
        db = client[db_name]
        collection = db[collection_name]
        
        # Build the query
        cursor = collection.find(
            filter=query or {},
            projection=projection
        )
        
        # Apply sorting if specified
        if sort:
            cursor = cursor.sort(sort)
            
        # Apply limit if specified
        if limit:
            cursor = cursor.limit(limit)
        
        # Convert cursor to list of documents
        results = list(cursor)
        
        # Convert ObjectId to string for JSON serialization
        for doc in results:
            if '_id' in doc and not isinstance(doc['_id'], (int, str, float)):
                doc['_id'] = str(doc['_id'])
        
        end_time = time.time()
        return {
            'data': results,
            'time': end_time - start_time,
            'row_count': len(results)
        }
    except Exception as e:
        print(f"ERROR: MongoDB query failed: {e}")
        return {
            'data': [],
            'time': time.time() - start_time,
            'row_count': 0,
            'error': str(e)
        }

In [17]:
def cassandra_query(session, query, params=None, fetch_size=None, keyspace='benchmark'):
    """
    Execute a CQL SELECT query on Cassandra.
    
    Args:
        session: Cassandra session object
        query: CQL query string
        params: Optional parameters for the query (list, tuple or dict)
        fetch_size: Number of rows to fetch at once (None for default)
        keyspace: Keyspace name
        
    Returns:
        dict: {'data': results, 'time': execution_time, 'row_count': count}
    """
    start_time = time.time()
    
    try:
        # Set the keyspace
        session.execute(f"USE {keyspace};")
        
        # Set fetch size if specified
        if fetch_size:
            session.default_fetch_size = fetch_size
            
        # Execute query
        if params:
            rows = session.execute(query, params)
        else:
            rows = session.execute(query)
            
        # Convert to list of dicts
        results = []
        for row in rows:
            # Convert Row to dict and handle any non-serializable types
            row_dict = {}
            for key in row._fields:
                value = getattr(row, key)
                # Convert non-serializable types
                if isinstance(value, (datetime.datetime, datetime.date)):
                    value = value.isoformat()
                elif isinstance(value, uuid.UUID):
                    value = str(value)
                row_dict[key] = value
            results.append(row_dict)
            
        end_time = time.time()
        return {
            'data': results,
            'time': end_time - start_time,
            'row_count': len(results)
        }
    except Exception as e:
        print(f"ERROR: Cassandra query failed: {e}")
        return {
            'data': [],
            'time': time.time() - start_time,
            'row_count': 0,
            'error': str(e)
        }

In [18]:
import time
import psutil
import statistics
import contextlib
import functools
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict, Any, Callable, Optional, Union
from datetime import datetime

class DatabaseBenchmark:
    """
    Comprehensive benchmarking wrapper for database operations that measures:
    - Total execution time
    - Individual operation times
    - Throughput (ops/sec)
    - System resource usage (CPU, RAM, I/O)
    """
    
    def __init__(self, benchmark_name: str, db_type: str, scale: int = 100):
        """
        Initialize benchmark settings
        
        Args:
            benchmark_name: Name of the benchmark scenario
            db_type: Type of database (postgres, mariadb, mongodb, cassandra)
            scale: Data scale factor (100, 1000, etc.)
        """
        self.benchmark_name = benchmark_name
        self.db_type = db_type
        self.scale = scale
        self.results = {
            'operations': [],
            'durations': [],
            'timestamps': [],
            'cpu_samples': [],
            'memory_samples': [],
            'disk_read_samples': [],
            'disk_write_samples': [],
            'start_time': None,
            'end_time': None
        }
        self.process = psutil.Process()
        self.io_counters_start = None
        
    def start(self):
        """Start the benchmark and initialize resource tracking"""
        self.results['start_time'] = time.time()
        self.io_counters_start = psutil.disk_io_counters()
        return self
    
    def end(self):
        """End the benchmark and calculate final statistics"""
        self.results['end_time'] = time.time()
        
        # Calculate summary statistics
        total_time = self.results['end_time'] - self.results['start_time']
        op_count = len(self.results['operations'])
        
        if op_count > 0:
            avg_time = statistics.mean(self.results['durations'])
            throughput = op_count / total_time
            
            # Calculate resource usage statistics
            cpu_avg = statistics.mean(self.results['cpu_samples']) if self.results['cpu_samples'] else 0
            memory_avg = statistics.mean(self.results['memory_samples']) / (1024 * 1024) if self.results['memory_samples'] else 0  # MB
            
            # Disk I/O statistics
            io_counters_end = psutil.disk_io_counters()
            read_bytes = io_counters_end.read_bytes - self.io_counters_start.read_bytes
            write_bytes = io_counters_end.write_bytes - self.io_counters_start.write_bytes
            
            # Store summary statistics
            self.summary = {
                'benchmark_name': self.benchmark_name,
                'db_type': self.db_type,
                'scale': self.scale,
                'total_time': total_time,
                'operation_count': op_count,
                'avg_operation_time': avg_time,
                'throughput': throughput,
                'cpu_avg': cpu_avg,
                'memory_avg_mb': memory_avg,
                'disk_read_mb': read_bytes / (1024 * 1024),
                'disk_write_mb': write_bytes / (1024 * 1024),
            }
        else:
            self.summary = {
                'benchmark_name': self.benchmark_name,
                'db_type': self.db_type,
                'scale': self.scale,
                'total_time': total_time,
                'operation_count': 0,
                'avg_operation_time': 0,
                'throughput': 0
            }
        
        return self.summary
    
    def record_operation(self, name: str, duration: float):
        """Record a single database operation with its duration"""
        timestamp = time.time()
        self.results['operations'].append(name)
        self.results['durations'].append(duration)
        self.results['timestamps'].append(timestamp)
        
        # Sample resource usage
        self.results['cpu_samples'].append(self.process.cpu_percent())
        self.results['memory_samples'].append(self.process.memory_info().rss)
        
        # Return the duration for convenience
        return duration
    
    @contextlib.contextmanager
    def measure_operation(self, name: str):
        """Context manager to measure a database operation"""
        start_time = time.time()
        try:
            yield
        finally:
            duration = time.time() - start_time
            self.record_operation(name, duration)
    
    def wrap_function(self, func: Callable, operation_name: str = None):
        """
        Decorator to wrap a function for benchmarking
        """
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            op_name = operation_name or func.__name__
            start_time = time.time()
            result = func(*args, **kwargs)
            duration = time.time() - start_time
            self.record_operation(op_name, duration)
            return result
        return wrapper
    
    def generate_report(self, output_file: Optional[str] = None) -> pd.DataFrame:
        """Generate a detailed report of the benchmark results"""
        if not hasattr(self, 'summary'):
            self.end()  # Ensure summary is calculated
            
        # Create DataFrame for detailed operation timings
        df = pd.DataFrame({
            'operation': self.results['operations'],
            'duration': self.results['durations'],
            'timestamp': self.results['timestamps'],
        })
        
        # Add relative timestamps
        if len(df) > 0:
            df['relative_time'] = df['timestamp'] - self.results['start_time']
        
        # Print summary to console
        print(f"\n{'=' * 80}")
        print(f"BENCHMARK SUMMARY: {self.benchmark_name} - {self.db_type} (Scale: {self.scale})")
        print(f"{'-' * 80}")
        print(f"Total execution time:      {self.summary['total_time']:.4f} seconds")
        print(f"Number of operations:      {self.summary['operation_count']} operations")
        print(f"Average operation time:    {self.summary['avg_operation_time']:.4f} seconds")
        print(f"Throughput:                {self.summary['throughput']:.2f} operations/second")
        print(f"CPU usage (average):       {self.summary['cpu_avg']:.2f}%")
        print(f"Memory usage (average):    {self.summary['memory_avg_mb']:.2f} MB")
        print(f"Disk read:                 {self.summary['disk_read_mb']:.2f} MB")
        print(f"Disk write:                {self.summary['disk_write_mb']:.2f} MB")
        print(f"{'=' * 80}\n")
        
        # Save to file if requested
        if output_file:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"{output_file}_{self.db_type}_{self.scale}_{timestamp}.csv"
            df.to_csv(filename, index=False)
            print(f"Detailed results saved to: {filename}")
            
            # Also save summary
            summary_df = pd.DataFrame([self.summary])
            summary_filename = f"{output_file}_{self.db_type}_{self.scale}_{timestamp}_summary.csv"
            summary_df.to_csv(summary_filename, index=False)
            print(f"Summary saved to: {summary_filename}")
        
        return df

# Example usage functions for each database type
def run_benchmark_scenario(db_type, scale, operations):
    """
    Run a complete benchmark scenario, measuring all operations
    
    Args:
        db_type: Database type (postgres, mariadb, mongodb, cassandra)
        scale: Data scale factor
        operations: List of operation functions to run
        
    Returns:
        DatabaseBenchmark object with results
    """
    benchmark = DatabaseBenchmark(f"Complete-Scenario", db_type, scale).start()
    
    try:
        for op_name, op_func in operations:
            with benchmark.measure_operation(op_name):
                op_func()
    finally:
        benchmark.end()
        benchmark.generate_report()
    
    return benchmark

In [ ]:
def postgres_benchmark_example(postgres_client, student_id=1):
    """Run a sample benchmark for PostgreSQL operations"""
    
    # Define operations to benchmark
    def op1():
        # Simple student query
        return postgres_query(
            postgres_client,
            "SELECT * FROM students WHERE id = %s",
            [student_id]
        )
        
    # Create operations list
    operations = [("Student Query", op1) for i in range(10000)]
    
    
    # Run benchmark
    return run_benchmark_scenario("postgres", 100, operations)
        
postgres_benchmark_example(postgres_client, student_id=1)


BENCHMARK SUMMARY: Complete-Scenario - postgres (Scale: 100)
--------------------------------------------------------------------------------
Total execution time:      1.4843 seconds
Number of operations:      10000 operations
Average operation time:    0.0001 seconds
Throughput:                6737.04 operations/second
CPU usage (average):       22.30%
Memory usage (average):    151.26 MB
Disk read:                 1.88 MB
Disk write:                0.48 MB

